In [1]:
import tensorflow as tf
import fastText as ft
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine as cosine_dist
from pprint import pprint
from tqdm import tqdm

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model = ft.load_model("../embeddings/fasttext/wiki.en.bin")

In [3]:
db = pd.read_csv('dataset/dataset.csv')


In [4]:
def get_embedding(model, corpus):
    """ For now just based on sum of all words embedings"""
    corpus = corpus.replace('\n', '').split()
    a = [0] * len(model.get_word_vector('0'))
    for w in corpus:
        a += model.get_word_vector(w)/len(corpus)
    return a


    

In [5]:
emb_db = []
for i in range(db['text'].count()):
    corpus = db['text'][i] #.replace('\n', '').split()
    emb_db.append(get_embedding(model, corpus))
    
pd.DataFrame(emb_db).to_csv('database/song_embedings.csv')

In [6]:
sentence = "Single ladies"
ind = []
for song in emb_db:
    ind.append(cosine_dist(get_embedding(model,sentence), song))
    
ind = np.argmax(ind)


In [7]:
print(sentence)
print(db.ix[ind]['song'])
print(db.ix[ind]['artist'])
pprint(db.ix[ind]['text'])

Single ladies
Answer Me
Frankie Laine
('ANSWER ME OH MY LOVE  \n'
 'JUST WHAT SIN HAVE I BEEN GUILTY OF  \n'
 'TELL ME HOW I CAME TO LOSE YOUR LOVE  \n'
 'PLEASE ANSWER ME MY LOVE  \n'
 '  \n'
 'YOU WERE MINE YESTERDAY  \n'
 'I BELIEVED THAT LOVE WAS HERE TO STAY  \n'
 "WON'T YOU TELL ME WHERE I'VE GONE ASTRAY  \n"
 'PLEASE ANSWER ME MY LOVE  \n'
 '  \n'
 "IF YOU'RE HAPPIER WITHOUT ME  \n"
 "I'LL TRY NOT TO CARE  \n"
 'BUT IF YOU STILL THINK ABOUT ME  \n'
 'PLEASE LISTEN TO MY PRAYER  \n'
 '  \n'
 "YOU MUST KNOW I'VE BEEN TRUE  \n"
 "WON'T YOU SAY THAT WE CAN START ANEW  \n"
 'IN MY SORROW NOW I TURN TO YOU  \n'
 'PLEASE ANSWER ME MY LOVE\n'
 '\n')


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [8]:
from tensorflow.contrib.rnn import LSTMCell, DropoutWrapper, MultiRNNCell


In [9]:
# TENSORFLOW STUFF STARTS HERE

seq_len = 1
gamma = 0.95  # Discount rate for future actions (Bellman equation)
epsilon = 1.0  # Exploration rate (E-greedy)
epsilon_min = 0.01
learning_rate = 0.001
dropout = 0.5

INPUT_DIM = 300  # EMBEDING SIZE (300?) ?
OUTPUT_DIM = 300 

graph = tf.Graph()
with graph.as_default():    



    # Define the input data of the model
    inputs = tf.placeholder(tf.float32, shape=(None, None, INPUT_DIM))
    targets = tf.placeholder(tf.float32, shape=(None, OUTPUT_DIM))
    batch_size = 1 # tf.shape(inputs)[1]


    def dropout_lstm(n=100, dropout=0.5):
        lstm = LSTMCell(n, state_is_tuple=True)
        lstm_drop = DropoutWrapper(lstm, output_keep_prob= 1.0 - dropout)
        return lstm_drop

    # Mulity layered RNN by LSTM cells
    with tf.name_scope('lstm'):
        lstm_stack = MultiRNNCell([dropout_lstm(i) for i in [1000,150,50]]) # Fucking chosen at random
        initial_state = lstm_stack.zero_state(batch_size=batch_size, dtype=tf.float32)

    with tf.name_scope('rnn'):
        outputs, final_state = tf.nn.dynamic_rnn(lstm_stack, inputs, initial_state = initial_state)
        outputs = outputs[:,-1] # We only need the last output tensor to pass into the fully connected layer

    with tf.name_scope('dense'):
        predictions = tf.layers.dense(outputs, OUTPUT_DIM)

    with tf.name_scope('loss'):
        # Cost function and optimizer
        cost = tf.losses.mean_squared_error(predictions, targets)

    # Gradient clipping
    with tf.name_scope('train'):
        train_op = tf.train.AdamOptimizer(learning_rate)
        gradients = train_op.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
        optimizer = train_op.apply_gradients(capped_gradients)

    tf.summary.scalar("cost", cost)

    # merge all summaries into a single "operation" which we can execute in a session 
    summary_op = tf.summary.merge_all()

    # Tensorboard summary writter
    writer = tf.summary.FileWriter('log', graph=tf.get_default_graph())

    # Variables initializer
    init_op = tf.global_variables_initializer()


In [10]:
# cld2


In [11]:
def get_single_embeding(model, corpus):
    """ For now just based on sum of all words embedings"""
    corpus = corpus.replace('\n', '').split()
    a = []
    for w in corpus:
        a.append(model.get_word_vector(w))
    return a
        
down = db.shape[0]
for j in tqdm(range(200)):
    text = db['text'][j]

    corpus = get_single_embeding(model, text)
    # saver = tf.train.Saver() -> TO DO: Implement model saver/loader
    sess = tf.Session(graph=graph)
    sess.run(init_op)

#     print(len(corpus))
    for i in range(len(corpus)-1):
        word = [corpus[:-1]]
        target_word = [corpus[1:]]
        _, summary = sess.run([optimizer, summary_op], feed_dict={inputs : word, targets : target_word})
#         print(sess.run([cost],  feed_dict={inputs : word, targets : target_word}))
        writer.add_summary(summary, i)
#     a = sess.run([final_state],  feed_dict={inputs : word, targets : target_word})
#     if j % 100:
#         print(sess.run([cost],  feed_dict={inputs : word, targets : target_word}))

  0%|          | 0/57650 [00:00<?, ?it/s]


ValueError: Cannot feed value of shape (1, 152, 300) for Tensor 'Placeholder_1:0', which has shape '(?, 300)'

In [ ]:
to_find = input()

word = [[to_find[:-1]]]
target_word = [to_find[1:]]
baseline_check = sess.run([final_state], feed_dict={inputs : word, targets : target_word})

IND = []
for i in range(len(corpus)-1):
    word = [[corpus[i]]]
    target_word = [corpus[i+1]]
    final_state = sess.run([final_state], feed_dict={inputs : word, targets : target_word})
    IND.append(cosine_dist(get_embedding(model,sentence), song))

In [ ]:
ind = np.argmax(IND)
print(to_find)
print(db.ix[ind]['song'])
print(db.ix[ind]['artist'])
pprint(db.ix[ind]['text'])